# 3분 딥러닝 3장 실습

# 20171130 목
p.35~38

작성된 소스 코드의 출처 : https://github.com/golbin/TensorFlow-Tutorials

In [2]:
import tensorflow as tf

In [3]:
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


tf.constant의 반환값은 Tensor()라고 해서 자료형과 속성에 대해 설명한다.

In [3]:
3

3

In [4]:
[1., 2., 3.]

[1.0, 2.0, 3.0]

In [5]:
[[1., 2., 3.], [4., 5., 6.]]

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

텐서는 텐서플로에서 수학식 계산을 위한 가장 기본적인 자료형.


랭크 = '\['의 갯수, 즉 차원 수. 
(예) 3 : 랭크 0, [3]: 랭크 1, [[3]]: 랭크 2, [[[3]]]: 랭크 3

랭크 0부터 순서대로, 스칼라, 벡터, 행렬, ... n차원 텐서라고 부른다.

셰이프는 랭크 별 원소 개수.
(예) 3 : 랭크 0 셰이프 \[\], \[3\] : 랭크 1 셰이프 \[1\], [[[1., 2., 3.]],[[1., 2., 3.]]] : 랭크 3 셰이프 [2, 1, 3]

In [6]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


텐서플로우 프로그램 구조 : 1. 그래프 생성(동그라미, tf.constant, tf.add, ...) 2. 그래프 실행(Session run). 이런 실행 방식을 '지연 실행' 또는 'Lazy Evaluation'이라고 부른다.

In [8]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close()

Hello, TensorFlow!
[10, 32, 42]


그래프의 실행은 세션 안에서 이루어져야 한다. 아래는 전체 코드

In [9]:
import tensorflow as tf

hello = tf.constant('Hello, TensorFlow!')
print(hello)

a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
print(c)

sess = tf.Session()
print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close()

Tensor("Const_3:0", shape=(), dtype=string)
Tensor("Add_1:0", shape=(), dtype=int32)
Hello, TensorFlow!
[10, 32, 42]


정리 : 텐서플로우는 그래프 기반 머신러닝 라이브러리이다. 프로그램 실행은 그래프 생성, 그래프 실행 순으로 진행된다. 랭크는 \[의 개수이며 차원 수를 의미한다. 랭크 0부터 스칼라, 벡터, 행렬, ... n차원 텐서라고 부른다. 셰이프는 각 랭크 별 원소 수를 뜻한다. [제일 큰 랭크의 원소수, 그 다음 랭크의 원소 수, ... ]로 적힌다.

# 20171206 수
<br>
p.38~43
<br>
플레이스홀더와 변수
플레이스홀더 : 그래프에 입력 값을 담을 그릇(parameter)을 담당.
변수(variable) : 학습 함수들이 학습 결과를 갱신하기 위해 사용하는 변수(가중치).

In [4]:
# None : 크기가 정해져 있지 않다.
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


In [5]:
x_data = [[1,2,3],[1,2,3]]

In [6]:
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

tf.random_normal은 정규 분포를 따르는 무작위 값으로 초기화한다는 뜻.

In [7]:
expr = tf.matmul(X, W) + b

[2,3]행렬 = 행이 2, 열이 3
<br>
(예) [[1,2,3],[1,2,3]] : [2, 3] 행렬

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict={X: x_data}))

sess.close()

=== x_data ===
[[1, 2, 3], [1, 2, 3]]
=== W ===
[[ 0.51939726  0.68138808]
 [ 1.71464252 -0.77968597]
 [ 0.19617501 -0.22691396]]
=== b ===
[[ 0.66313279]
 [ 0.87909079]]
=== expr ===
[[ 5.20034027 -0.89559305]
 [ 5.41629839 -0.67963505]]


placeholder가 session에서 돌아가려면 feed_dict를 통해 입력 데이터를 별도로 넣어야 에러가 나지 않는다.
<br>
애초에 placeholder는 그릇이니까 내용물을 feed_dict로 담는 건 자연스러운 일이다.

# 20171207 목

p. 44~
<br>
선형 회귀 모델 구현하기

X,Y -> 관계(직선 또는 곡선)
<br>
X -관계(직선 또는 곡선)-> Y
<br>
머신러닝은 이렇게 X, Y의 관계를 찾아내서
<br>
새로운 X를 넣었을 때 Y를 예측하는 것을 기본으로 한다.
<br>
<br>
균등 분포란 주사위를 던지는 확률이 모두 1/6이듯이 사건이 일어날 확률이 모두 동일한 확률 분포를 뜻한다.
<br>
손실 함수, loss function은 한 쌍의 데이터(x,y)에 대한 손실값을 계산하는 함수이다.
<br>
손실값이란 예측 값과 실제 값 사이의 차이를 뜻한다. 이 차이를 최소화할 수 있다면 예측도는 최대가 될 것이다.
<br>
손실값은 각 데이터에 대한 오차를 뜻하고, 비용(Cost)는 전체 데이터에 대한 손실을 뜻한다. 즉 각 손실값 등을 평균하는 것 등으로 구할 수 있다.
<br>
학습이란 W, b와 같은 변수에 다양한 값을 넣고 계산해서 손실값을 계산하고, 이 손실을 최소화하는 W, b를 구하는 과정이다.
<br>
손실 함수 중 대표적인 것이 예측 값 - 실제 값이다. 즉 차이, 거리를 뜻한다.
<br>
최소제곱 함수라고 하여서 이 차이에다가 제곱을 씌운 손실 함수를 아래 코드에 사용했다.
<br>
그리고 비용을 계산하기 위해 각 손실값들을 모두 더한 후 평균을 내는 reduce_mean을 사용했다. 여기서 reduce란 여러 개를 더해서 한 개의 숫자로 결과가 나오기 때문에 붙인 말로 보인다.
<br>
그렇다면 어떻게 비용을 최소화해야할까? 최적화 함수란 걸 쓰면 된다. 가장 유명한 게 경사하강법, Gradient Descent이다. 이는 손실함수의 미분값(변화량)을 변수에 반영(학습률을 곱해서 통제) 반영해서 손실값을 줄이는 새로운 변수를 갱신하는 방식을 택한다. 따라서 최적화함수를 많이 작동시키면 그 만큼 변수가 갱신될 것이고, 손실값도 점점 줄어들 것이다. (오버피팅은 일단 논외로)

In [14]:
import tensorflow as tf

x_data = [1, 2, 3]
y_data = [1, 2, 3]

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

hypothesis = W * X + b # W와 X까 헁렬이 아니므로 matmul을 사용하지 않음.

cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(100):
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data,
                                                           Y: y_data})
        print(step, cost_val, sess.run(W), sess.run(b))
    
    print("\n=== Test ===")
    print("X: 5, Y:",sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5 Y:",sess.run(hypothesis, feed_dict={X: 2.5}))

(0, 0.041261163, array([ 0.78460103], dtype=float32), array([ 0.48052654], dtype=float32))
(1, 0.033404078, array([ 0.79342943], dtype=float32), array([ 0.47058082], dtype=float32))
(2, 0.031746905, array([ 0.79799628], dtype=float32), array([ 0.45909286], dtype=float32))
(3, 0.030238071, array([ 0.80289596], dtype=float32), array([ 0.44807577], dtype=float32))
(4, 0.028801749, array([ 0.80762941], dtype=float32), array([ 0.43730223], dtype=float32))
(5, 0.027433641, array([ 0.81225443], dtype=float32), array([ 0.42679003], dtype=float32))
(6, 0.026130505, array([ 0.81676763], dtype=float32), array([ 0.41653025], dtype=float32))
(7, 0.024889275, array([ 0.82117242], dtype=float32), array([ 0.40651715], dtype=float32))
(8, 0.023707038, array([ 0.82547134], dtype=float32), array([ 0.39674476], dtype=float32))
(9, 0.022580942, array([ 0.82966685], dtype=float32), array([ 0.38720727], dtype=float32))
(10, 0.021508316, array([ 0.83376157], dtype=float32), array([ 0.37789908], dtype=float32)

# 20171208금
p.52~<br>
# Ch.4 기본 신경망 구현
(이어서 해야 함)
# 20171210 일
(이어서 계속)
p.62 ~<br>
인공 신경망은 뉴런의 구조에서 영감을 받은 구조로 가중치+활성화 함수의 연결로 구성되어 있다.<br>
가장 좋은 결과를 내기 위해 가중치를 일일이 찾을 수고를 제한된 볼츠만, 각종 활성화 함수, 역전파를 통해 찾을 수 있게 되었고, <br>
데이터 수의 기하급수 증가, GPU에 의한 컴퓨팅 파워 증가로 인해 신경망 학습은 딥러닝이라는 이름 아래 최고의 호황을 누리고 있다. <br>
역전파는 결과값과 실제값의 오차를 각 신경망에 역으로 넣어 계산해서 최종적으로 입력층까지 계산하는 방법이다. 이를 통해서 가중치를 수작업으로 한땀한땀 아닌 하나의 알고리즘으로써 유의미한 값을 갱신할 수 있게 해주었다. <br>
<br>
아래는 포유류와 조류를 구분하는 분류기 코드이다. <br>
심층 신경망은 다음 시간에 <br>

In [14]:
import tensorflow as tf
import numpy as np

In [15]:
# [털, 날개]
x_data = np.array(
[[0, 0],[1, 0],[1, 1],[0, 0],[0, 0],[0, 1]])

In [16]:
y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [17]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [18]:
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [23]:
L = tf.add(tf.matmul(X, W),b)
L = tf.nn.relu(L)

In [24]:
model = tf.nn.softmax(L)

In [25]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

axis = 1은 행 별로 더 하라는 뜻이다.<br>
더한다는 것은 차원을 축소한다는 것과 동일하다. 

In [26]:
# 기본적인 경사하강법으로 최적화합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y:y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step+1, sess.run(cost, feed_dict = {X:x_data, Y:y_data}))

(10, 1.1395766)
(20, 1.1383139)
(30, 1.1370722)
(40, 1.1358509)
(50, 1.1346496)
(60, 1.1334683)
(70, 1.1323062)
(80, 1.1311632)
(90, 1.1300391)
(100, 1.1289333)


In [27]:
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print("예측값:", sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

('\xec\x98\x88\xec\xb8\xa1\xea\xb0\x92:', array([0, 0, 0, 0, 0, 1]))
('\xec\x8b\xa4\xec\xa0\x9c\xea\xb0\x92:', array([0, 1, 2, 0, 0, 2]))


In [29]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict = {X:x_data, Y:y_data}))

정확도: 50.00


# 20171211 월
p.68~<br>
심층 신경망 구현

In [32]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
[[0, 0],[1, 0],[1, 1],[0, 0],[0, 0],[0, 1]])

y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10],-1.,1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1),b1)
L1 = tf.nn.relu(L1)

model = tf.add(tf.matmul(L1, W2),b2)

cost = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y:y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step+1, sess.run(cost, feed_dict = {X:x_data, Y:y_data}))

prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print("예측값:", sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict = {X:x_data, Y:y_data}))

(10, 0.97937375)
(20, 0.79200846)
(30, 0.61869645)
(40, 0.4685922)
(50, 0.35646558)
(60, 0.26688138)
(70, 0.19452591)
(80, 0.13982087)
(90, 0.10059301)
(100, 0.07326252)
('\xec\x98\x88\xec\xb8\xa1\xea\xb0\x92:', array([0, 1, 2, 0, 0, 2]))
('\xec\x8b\xa4\xec\xa0\x9c\xea\xb0\x92:', array([0, 1, 2, 0, 0, 2]))
정확도: 100.00


# 20171212 화

# Ch 5. 텐서보드와 모델 재사용

(내일 예정)